# Named Entity Recognition (NER) using BERT

In [2]:
import os
import pandas as pd
import numpy as np
import json
import re
from nltk.tokenize import sent_tokenize
from transformers import BertTokenizer, AutoTokenizer
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import transformers
from tqdm import tqdm
import glob
from sklearn.model_selection import train_test_split
import datetime
import warnings
warnings.filterwarnings('ignore')


## Model Config

In [6]:
model_name = 'bert-ner-token-classification.bin'
bert_path = 'bert-base-uncased'
train_path = './data/train/'
test_path = './data/test'

In [7]:
config = {
    'MAX_LEN': 128,
    'tokenizer': AutoTokenizer.from_pretrained(bert_path, do_lower_case=True),
    'batch_size': 5,
    'Epoch': 1, 
    'train_path': train_path,
    'test_path': test_path,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'model_path': model_name,
    'model_name': model_name
}

In [8]:
print(config)

{'MAX_LEN': 128, 'tokenizer': PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), 'batch_size': 5, 'Epoch': 1, 'train_path': './data/train/', 'test_path': './data/test', 'device': 'cuda', 'model_path': 'bert-ner-token-classification.bin', 'model_name': 'bert-ner-token-classification.bin'}


## Read the train data and combine data labels